In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Make sure to convert categorical data to numbers. Use 80% of the data as the train_dataset and 20% of the data as the test_dataset.

# convert categorical columns to numerical data
categorical_cols = ['sex', 'smoker', 'region']
for col in categorical_cols:
    dataset[col], _ = dataset[col].factorize()

# Use 80% of the data for train_dataset and the remainder for test_dataset
train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

# pop off the "expenses" column from these datasets to create new datasets called train_labels and test_labels. Use these labels when training your
# model

train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')


In [ ]:
# Create a model and train it with the train_dataset. Run the final cell in this notebook to check your model. The final cell will use the
# unseen test_dataset to check how well the model generalizes

# normalize training data
normalizer = layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_dataset))

# create a sequential model
model = tf.keras.Sequential([
    # normalize the data
    normalizer,
    # first layer with 6 units and ReLU activation
    layers.Dense(units=6, activation='relu'),
    # second layer with 6 units and ReLU activation
    layers.Dense(units=6, activation='relu'),
    # output layer with 1 unit for regression
    layers.Dense(units=1)
])

# compile the model
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error',
    metrics=['mae', 'mse'])

model.summary()

In [ ]:
# Fit model and get training history
history = model.fit(
    # Dataset and labels to use for training
    train_dataset, # training dataset
    train_labels, # training labels
    batch_size=32,
    epochs=50,
    verbose=0,
    validation_split = 0.2)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
